In [1]:
import pandas as pd
from pathlib import Path

In [2]:
pd.read_csv('/media/t9s9/SSD_ubuntu/datasets/AudioSet/original_annot/balanced_train_segments.csv')

ParserError: Error tokenizing data. C error: Expected 4 fields in line 4, saw 5


In [3]:
idx_lab = pd.read_csv('/media/t9s9/SSD_ubuntu/datasets/AudioSet/class_labels_indices.csv')
val_df = pd.read_csv('/media/t9s9/SSD_ubuntu/datasets/AudioSet/original_annot/eval_segments.csv')
train_df = pd.read_csv('/media/t9s9/SSD_ubuntu/datasets/AudioSet/audioset_train.csv')

In [5]:
train_df.shape

(2041789, 4)

In [66]:
idx_to_label = idx_lab.drop(columns=['index']).set_index('mid').to_dict()['display_name']
apply_list = lambda x: [idx_to_label[i.strip('"')] for i in x]

In [55]:
val_df['labels'] = val_df[' positive_labels'].apply(lambda x: apply_list(x.split(',')))
val_df = val_df.drop(columns=['time_start', 'time_end', ' positive_labels', 'youtube_id'])
val_df['filename'] = val_df['filename'].apply(lambda x: str(Path('val', x)))
exists = val_df['filename'].apply(lambda x: Path('/home/t9s9/Datasets/AudioSet/mp3/', x).with_suffix('.mp3').exists())
val_df = val_df[exists]
val_df.to_parquet('/home/t9s9/Datasets/AudioSet/annot_val.parquet', index=False)

In [82]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1738510 entries, 0 to 2041788
Data columns (total 2 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   labels    object
 1   filename  object
dtypes: object(2)
memory usage: 39.8+ MB


In [67]:
train_df['labels'] = train_df['positive_labels'].apply(lambda x: apply_list(x.split(',')))

In [74]:
train_df['filename'] = train_df.apply(lambda row: f"train/{row['youtube_id']}_{int(row['time_start']):06d}_{int(row['time_end']):06d}.mp4", axis=1)

In [76]:
exists = train_df['filename'].apply(lambda x: Path('/home/t9s9/Datasets/AudioSet/mp3/', x).with_suffix('.mp3').exists())

In [79]:
train_df = train_df[exists]

In [80]:
train_df = train_df.drop(columns=['time_start', 'time_end', 'positive_labels', 'youtube_id'])

In [83]:
train_df.to_parquet('/home/t9s9/Datasets/AudioSet/annot_train.parquet', index=False)

# Prepare Download

In [3]:
df = pd.read_csv('/media/ts/SSD_ubuntu/datasets/AudioSet/original_annot/unbalanced_train_segments.csv',
                 skiprows=2, sep=', ', engine='python')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2041789 entries, 0 to 2041788
Data columns (total 4 columns):
 #   Column           Dtype  
---  ------           -----  
 0   # YTID           object 
 1   start_seconds    float64
 2   end_seconds      float64
 3   positive_labels  object 
dtypes: float64(2), object(2)
memory usage: 62.3+ MB


In [4]:
df = df.rename(columns={'# YTID': 'youtube_id',
                        'start_seconds':'time_start',
                        'end_seconds':'time_end'
                        })
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2041789 entries, 0 to 2041788
Data columns (total 4 columns):
 #   Column           Dtype  
---  ------           -----  
 0   youtube_id       object 
 1   time_start       float64
 2   time_end         float64
 3   positive_labels  object 
dtypes: float64(2), object(2)
memory usage: 62.3+ MB


In [5]:
df.to_csv('/media/ts/SSD_ubuntu/datasets/AudioSet/audioset_train.csv', index=False)

In [15]:
df = pd.read_parquet('/home/t9s9/Datasets/AudioSet/annot_full_val.parquet')
df_cls = pd.read_parquet('/home/t9s9/Datasets/AudioSet/annot_val.parquet')

In [16]:
df = df.merge(df_cls, on='filename', how='inner', validate='one_to_one')

In [17]:
df = df.rename(columns={'labels':'tags'})

In [18]:
df

,filename,captions_visual,captions_auditory,tags
0,val/--4gqARaEJE_000000_000010.mp4,[Puppy that looks like a dachshund - photo #.],[A small dog whimpering.],"[Domestic animals, pets, Squeak, Dog, Animal]"
1,val/--BfvyPmVMo_000020_000030.mp4,[How to fix a flat tire on a car.],[A drill is running and a man speaks.],[Hammer]
2,val/--U7joUcTCo_000000_000010.mp4,[An elderly man with glasses smiles into the c...,[Laughter followed by laughter.],[Cough]
3,val/--i-y1v8Hy8_000000_000009.mp4,[Person shared a video of herself singing into...,[A group of people talking followed by a child...,"[Music, Speech, Female singing, Child singing]"
4,val/-0BIyqJj9ZU_000030_000040.mp4,[Funny : the video shows a man laughing as he ...,[A baby is laughing and a man is talking.],"[Chuckle, chortle, Belly laugh, Baby laughter]"
...,...,...,...,...
17024,val/zxuey_Zi0wY_000030_000040.mp4,[A man plays the accordion in the living room ...,[A musical instrument is being played.],"[Wind instrument, woodwind instrument, Accordion]"
17025,val/zyF8TGSRvns_000150_000160.mp4,[How to make a gift box for your child.],[Music is playing in the background as a violi...,"[Marimba, xylophone, Glockenspiel, Chime, Mall..."
17026,val/zz35Va7tYmA_000030_000040.mp4,[The video shows a man driving his car through...,[A car engine revving and accelerating.],"[Motor vehicle (road), Accelerating, revving, ..."
17027,val/zzNdwF40ID8_000070_000080.mp4,[How to draw a straight line through a triangle.],[Electronically generated electronic music.],"[Music, Sonar]"


In [19]:
df.to_parquet('/home/t9s9/Datasets/AudioSet/annot_full_val.parquet', index=False)